In [39]:
from sympy import *
from scipy.fftpack import rfft
import numpy as np
import matplotlib.pyplot as plt
init_printing()
# Analytical solution for the constant medium acoustic wave equation
# Given for a nx by ny domain of velocity v on a h grid with source at position xsrc,ysrc , 2second propagation at dt=1ms

T=2.0
dt=.001
df=1.0/T
maxf=1.0/dt
nt=floor(T/dt+1)

tt=np.linspace(0,T,nt)
ff=rfft(tt)
tt,ff

fig2 = plt.figure()
plt.hold("off")
shotrec = plt.plot(ff,ff)   # this is how you'd plot a single line...
plt.show()

numpy.ndarray

In [ ]:
nx=200
ny=200
h=1
xsrc=100 # in grid points
ysrc=100 # in grid points
v=1

U_a=np.zeros((nx,ny,nf))
for a in range(0,nf):
    k = 2*np.pi*f[a]/v

    for m in range(0,nx):
        for n in range(0,ny):
            tmp = k*sqrt((h*(m - xsrc))**2 + h*(n - ysrc)**2)
            U_a[m,n,a] = 1j* hankel1(0,tmp)/4.0;

In [ ]:
# Do inverse fft on 0:.001:2 (third dimension on U) and you have analytical solution o